In [17]:
from ctypes import *
import time
from sys import path
import sys
from os import sep
import pandas as pd
import matplotlib.pyplot as plt

if sys.platform.startswith("win"):
    dwf = cdll.dwf
    constants_path = "C:" + sep + "Program Files (x86)" + sep + "Digilent" + sep + "WaveFormsSDK" + sep + "samples" + sep + "py"
elif sys.platform.startswith("darwin"):
    dwf = cdll.LoadLibrary("/Library/Frameworks/dwf.framework/dwf")
    constants_path = "/Applications/WaveForms.app/Contents/Resources/SDK/samples/py"
else:
    dwf = cdll.LoadLibrary("libdwf.so")
    constants_path = "/usr/share/digilent/waveforms/samples/py"

path.append(constants_path)
import dwfconstants as constants

In [2]:
dwf.FDwfDeviceCloseAll()
filter_flags = c_int(constants.enumfilterType.value | constants.enumfilterUSB.value)
device_count = c_int()
dwf.FDwfEnum(filter_flags, byref(device_count))

hdwf = c_int()
dwf.FDwfDeviceOpen(c_int(-1), byref(hdwf))
if hdwf.value == 0:
    raise RuntimeError("Failed to open device.")
print(hdwf.value)

dwf.FDwfAnalogIOReset(hdwf)
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(1), c_double(3.0))
dwf.FDwfAnalogIOChannelNodeSet(hdwf, c_int(0), c_int(0), c_double(1))
dwf.FDwfAnalogIOEnableSet(hdwf, c_int(1))
time.sleep(0.5)

vpos = c_double()
dwf.FDwfAnalogIOStatus(hdwf)
dwf.FDwfAnalogIOChannelNodeStatus(hdwf, c_int(0), c_int(1), byref(vpos))
print(f"Power: +{vpos.value:.2f}V")

1
Power: +3.00V


In [3]:
DEVICE_ADDR = 0x52
ENABLE_REG = 0x80
ATIME_REG = 0x81
CONTROL_REG = 0x8F
CDATAL_REG = 0x94
CDATAH_REG = 0x95
RDATAL_REG = 0x96
RDATAH_REG = 0x97
GDATAL_REG = 0x98
GDATAH_REG = 0x99
BDATAL_REG = 0x9A
BDATAH_REG = 0x9B

def write_register(reg, value):
    iNak = c_int()
    rgTX = (c_ubyte * 2)(reg, value)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(2), byref(iNak))
    return iNak.value == 0

def read_register(reg):
    iNak = c_int()
    rgTX = (c_ubyte * 1)(reg)
    dwf.FDwfDigitalI2cWrite(hdwf, c_int(DEVICE_ADDR), rgTX, c_int(1), byref(iNak))
    if iNak.value == 0:
        rgRX = (c_ubyte * 1)()
        dwf.FDwfDigitalI2cRead(hdwf, c_int(DEVICE_ADDR), rgRX, c_int(1), byref(iNak))
        if iNak.value == 0:
            return rgRX[0]
    return None

dwf.FDwfDigitalI2cRateSet(hdwf, c_double(100e3))
dwf.FDwfDigitalI2cSclSet(hdwf, c_int(0))
dwf.FDwfDigitalI2cSdaSet(hdwf, c_int(1))
iNak = c_int()
dwf.FDwfDigitalI2cClear(hdwf, byref(iNak))

1

In [4]:
write_register(ENABLE_REG, 0x01)
time.sleep(0.01)
write_register(ENABLE_REG, 0x03)
write_register(ATIME_REG, 0xFF)
write_register(CONTROL_REG, 0x02)

def read_colors():
    time.sleep(0.1)
    
    clear_low = read_register(CDATAL_REG)
    clear_high = read_register(CDATAH_REG)
    red_low = read_register(RDATAL_REG)
    red_high = read_register(RDATAH_REG)
    green_low = read_register(GDATAL_REG)
    green_high = read_register(GDATAH_REG)
    blue_low = read_register(BDATAL_REG)
    blue_high = read_register(BDATAH_REG)
    
    if all(val is not None for val in [clear_low, clear_high, red_low, red_high, green_low, green_high, blue_low, blue_high]):
        clear = (clear_high << 8) | clear_low
        red = (red_high << 8) | red_low
        green = (green_high << 8) | green_low
        blue = (blue_high << 8) | blue_low
        return {'clear': clear, 'red': red, 'green': green, 'blue': blue}
    return None

In [5]:
data0={'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(CONTROL_REG,0x00) # Gain value 1
for i in range(100):
    colors = read_colors()
    data0['clear'].append(colors['clear'])
    data0['red'].append(colors['red'])
    data0['green'].append(colors['green'])
    data0['blue'].append(colors['blue'])
    print(f"Index {i}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Index 0: 2, R: 1, G: 1, B: 1
Index 1: 3, R: 1, G: 1, B: 1
Index 2: 2, R: 0, G: 1, B: 1
Index 3: 3, R: 0, G: 1, B: 1
Index 4: 3, R: 1, G: 1, B: 1
Index 5: 3, R: 1, G: 1, B: 1
Index 6: 2, R: 0, G: 1, B: 1
Index 7: 2, R: 0, G: 1, B: 1
Index 8: 3, R: 1, G: 1, B: 1
Index 9: 3, R: 1, G: 1, B: 1
Index 10: 3, R: 1, G: 1, B: 1
Index 11: 3, R: 0, G: 1, B: 1
Index 12: 3, R: 1, G: 1, B: 1
Index 13: 2, R: 0, G: 1, B: 1
Index 14: 2, R: 1, G: 1, B: 1
Index 15: 3, R: 1, G: 1, B: 1
Index 16: 3, R: 1, G: 1, B: 1
Index 17: 3, R: 0, G: 1, B: 1
Index 18: 2, R: 1, G: 1, B: 1
Index 19: 2, R: 0, G: 1, B: 1
Index 20: 3, R: 1, G: 1, B: 1
Index 21: 2, R: 0, G: 1, B: 1
Index 22: 3, R: 1, G: 1, B: 1
Index 23: 3, R: 1, G: 1, B: 1
Index 24: 2, R: 1, G: 1, B: 1
Index 25: 3, R: 0, G: 1, B: 1
Index 26: 2, R: 1, G: 1, B: 1
Index 27: 2, R: 0, G: 1, B: 1
Index 28: 3, R: 1, G: 1, B: 1
Index 29: 3, R: 1, G: 1, B: 1
Index 30: 2, R: 1, G: 1, B: 1
Index 31: 3, R: 1, G: 1, B: 1
Index 32: 3, R: 1, G: 1, B: 1
Index 33: 4, R: 1, G

In [8]:
df0 = pd.DataFrame(data0, columns=['clear', 'red', 'green', 'blue'])
print(df0)

    clear  red  green  blue
0       2    1      1     1
1       3    1      1     1
2       2    0      1     1
3       3    0      1     1
4       3    1      1     1
..    ...  ...    ...   ...
95      4    1      2     2
96      6    1      2     2
97      6    1      2     2
98      6    1      2     2
99      4    1      2     2

[100 rows x 4 columns]


In [9]:
data1={'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(CONTROL_REG,0x01) # Gain value 4
for i in range(100):
    colors = read_colors()
    data1['clear'].append(colors['clear'])
    data1['red'].append(colors['red'])
    data1['green'].append(colors['green'])
    data1['blue'].append(colors['blue'])
    print(f"Index {i}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Index 0: 15, R: 4, G: 4, B: 4
Index 1: 15, R: 3, G: 5, B: 5
Index 2: 11, R: 4, G: 6, B: 5
Index 3: 17, R: 4, G: 4, B: 4
Index 4: 13, R: 3, G: 4, B: 4
Index 5: 12, R: 3, G: 5, B: 5
Index 6: 12, R: 4, G: 6, B: 5
Index 7: 17, R: 2, G: 4, B: 4
Index 8: 12, R: 3, G: 4, B: 6
Index 9: 14, R: 3, G: 5, B: 5
Index 10: 13, R: 3, G: 5, B: 5
Index 11: 11, R: 3, G: 4, B: 4
Index 12: 11, R: 3, G: 4, B: 4
Index 13: 15, R: 3, G: 6, B: 4
Index 14: 16, R: 4, G: 6, B: 5
Index 15: 17, R: 4, G: 6, B: 5
Index 16: 12, R: 3, G: 4, B: 4
Index 17: 14, R: 3, G: 5, B: 5
Index 18: 12, R: 3, G: 5, B: 4
Index 19: 11, R: 2, G: 4, B: 4
Index 20: 12, R: 3, G: 4, B: 4
Index 21: 15, R: 3, G: 6, B: 5
Index 22: 15, R: 3, G: 5, B: 5
Index 23: 11, R: 4, G: 4, B: 4
Index 24: 17, R: 3, G: 4, B: 6
Index 25: 13, R: 3, G: 4, B: 4
Index 26: 15, R: 3, G: 5, B: 5
Index 27: 11, R: 3, G: 4, B: 4
Index 28: 11, R: 2, G: 4, B: 4
Index 29: 13, R: 3, G: 4, B: 4
Index 30: 15, R: 3, G: 5, B: 5
Index 31: 12, R: 3, G: 5, B: 5
Index 32: 11, R: 3

In [13]:
df1 = pd.DataFrame(data1, columns=['clear', 'red', 'green', 'blue'])
print(df1)

    clear  red  green  blue
0      15    4      4     4
1      15    3      5     5
2      11    4      6     5
3      17    4      4     4
4      13    3      4     4
..    ...  ...    ...   ...
95     16    2      4     4
96     11    3      4     3
97     13    3      5     5
98     15    3      5     5
99     16    4      6     5

[100 rows x 4 columns]


In [12]:
data2={'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(CONTROL_REG, 0x02) # Gain value 16
for i in range(100):
    colors = read_colors()
    data2['clear'].append(colors['clear'])
    data2['red'].append(colors['red'])
    data2['green'].append(colors['green'])
    data2['blue'].append(colors['blue'])
    print(f"Index {i}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Index 0: 47, R: 11, G: 16, B: 15
Index 1: 58, R: 14, G: 19, B: 21
Index 2: 49, R: 12, G: 19, B: 20
Index 3: 69, R: 16, G: 25, B: 22
Index 4: 50, R: 11, G: 17, B: 15
Index 5: 60, R: 13, G: 21, B: 20
Index 6: 47, R: 11, G: 18, B: 17
Index 7: 46, R: 11, G: 16, B: 15
Index 8: 53, R: 15, G: 24, B: 22
Index 9: 54, R: 13, G: 20, B: 18
Index 10: 47, R: 11, G: 18, B: 22
Index 11: 69, R: 16, G: 25, B: 23
Index 12: 63, R: 15, G: 24, B: 22
Index 13: 53, R: 13, G: 20, B: 19
Index 14: 46, R: 11, G: 17, B: 16
Index 15: 69, R: 11, G: 16, B: 15
Index 16: 55, R: 12, G: 19, B: 22
Index 17: 52, R: 12, G: 21, B: 19
Index 18: 46, R: 11, G: 17, B: 16
Index 19: 48, R: 11, G: 17, B: 15
Index 20: 56, R: 13, G: 19, B: 22
Index 21: 50, R: 12, G: 19, B: 18
Index 22: 45, R: 11, G: 17, B: 22
Index 23: 67, R: 16, G: 25, B: 15
Index 24: 60, R: 14, G: 23, B: 22
Index 25: 51, R: 12, G: 23, B: 18
Index 26: 70, R: 11, G: 17, B: 16
Index 27: 65, R: 15, G: 24, B: 22
Index 28: 56, R: 13, G: 22, B: 18
Index 29: 68, R: 16, G: 

In [14]:
df2 = pd.DataFrame(data2, columns=['clear', 'red', 'green', 'blue'])
print(df2)

    clear  red  green  blue
0      47   11     16    15
1      58   14     19    21
2      49   12     19    20
3      69   16     25    22
4      50   11     17    15
..    ...  ...    ...   ...
95     48   11     25    23
96     59   13     20    18
97     68   15     23    21
98     70   16     25    23
99     66   12     17    16

[100 rows x 4 columns]


In [15]:
data3={'clear': [], 'red': [], 'green': [], 'blue': []}

write_register(CONTROL_REG,0x03) # Gain value 64
for i in range(100):
    colors = read_colors()
    data3['clear'].append(colors['clear'])
    data3['red'].append(colors['red'])
    data3['green'].append(colors['green'])
    data3['blue'].append(colors['blue'])
    print(f"Index {i}: {colors['clear']}, R: {colors['red']}, G: {colors['green']}, B: {colors['blue']}")

Index 0: 215, R: 57, G: 92, B: 86
Index 1: 257, R: 59, G: 88, B: 78
Index 2: 178, R: 42, G: 66, B: 62
Index 3: 189, R: 43, G: 65, B: 60
Index 4: 230, R: 56, G: 89, B: 84
Index 5: 197, R: 48, G: 76, B: 82
Index 6: 276, R: 64, G: 99, B: 90
Index 7: 196, R: 45, G: 67, B: 60
Index 8: 230, R: 52, G: 77, B: 69
Index 9: 190, R: 46, G: 73, B: 70
Index 10: 178, R: 42, G: 65, B: 61
Index 11: 257, R: 61, G: 96, B: 89
Index 12: 223, R: 52, G: 77, B: 82
Index 13: 191, R: 46, G: 74, B: 70
Index 14: 178, R: 64, G: 65, B: 90
Index 15: 259, R: 62, G: 97, B: 90
Index 16: 224, R: 54, G: 87, B: 82
Index 17: 265, R: 61, G: 74, B: 70
Index 18: 179, R: 42, G: 65, B: 62
Index 19: 194, R: 44, G: 66, B: 60
Index 20: 228, R: 57, G: 90, B: 85
Index 21: 200, R: 49, G: 89, B: 80
Index 22: 275, R: 42, G: 98, B: 62
Index 23: 192, R: 44, G: 66, B: 60
Index 24: 226, R: 57, G: 76, B: 67
Index 25: 258, R: 59, G: 89, B: 79
Index 26: 177, R: 42, G: 65, B: 61
Index 27: 188, R: 43, G: 96, B: 89
Index 28: 222, R: 54, G: 86, B

In [16]:
df3 = pd.DataFrame(data3, columns=['clear', 'red', 'green', 'blue'])
print(df3)

    clear   red  green  blue
0     215    57     92    86
1     257    59     88    78
2     178    42     66    62
3     189    43     65    60
4     230    56     89    84
..    ...   ...    ...   ...
95   1024  1024   1024  1024
96   1024  1024   1024  1024
97   1024   718    692   523
98    818   261    254   207
99    501   155    195   167

[100 rows x 4 columns]


In [35]:
fig, ax = plt.subplots()

# Plot each DataFrame on the same axes

df0['clear'].plot(ax=ax, label='AGAIN_1')
df1['clear'].plot(ax=ax, label='AGAIN_4')
df2['clear'].plot(ax=ax, label='AGAIN_16')
df3['clear'].plot(ax=ax, label='AGAIN_64')

# Add legend and display

plt.title('Gain Comparison of Clear Channel Readings')
plt.legend()
#plt.figure(figsize=(40,15))
plt.show()

In [24]:
fig, ax = plt.subplots()

# Plot each DataFrame on the same axes
df0['red'].plot(ax=ax, label='AGAIN_1')
df1['red'].plot(ax=ax, label='AGAIN_4')
df2['red'].plot(ax=ax, label='AGAIN_16')
df3['red'].plot(ax=ax, label='AGAIN_64')

# Add legend and display
plt.title('Gain Comparison of Red Channel Readings')
plt.legend()
plt.show()

In [25]:
fig, ax = plt.subplots()

# Plot each DataFrame on the same axes
df0['green'].plot(ax=ax, label='AGAIN_1')
df1['green'].plot(ax=ax, label='AGAIN_4')
df2['green'].plot(ax=ax, label='AGAIN_16')
df3['green'].plot(ax=ax, label='AGAIN_64')

# Add legend and display
plt.title('Gain Comparison of Green Channel Readings')
plt.legend()
plt.show()

In [26]:
fig, ax = plt.subplots()

# Plot each DataFrame on the same axes
df0['blue'].plot(ax=ax, label='AGAIN_1')
df1['blue'].plot(ax=ax, label='AGAIN_4')
df2['blue'].plot(ax=ax, label='AGAIN_16')
df3['blue'].plot(ax=ax, label='AGAIN_64')

# Add legend and display
plt.title('Gain Comparison of Blue Channel Readings')
plt.legend()
plt.show()

In [27]:
def cleanup():
    dwf.FDwfDeviceClose(hdwf)
cleanup()